In [10]:
# Fonction qui trouve une solution initiale realisable
function initial_column()
    SP=Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(SP, "OutputFlag", 0)
    
    # Definition x_a = 1 si l'arc a est dans le cycle retenu, 0 sinon
    @variable(SP,x[a in 1:ne(pool)],Bin)
    
    # Definition de y_i = 1 si paire i dans le cycle retenu, 0 sinon
    @variable(SP,y[i in 1:nv(pool)],Bin)
    
    
    for vertice in 1:nv(pool)
        @constraint(SP,sum(x[a] for a in origine[vertice]) == sum(x[a] for a in dest[vertice]) )
        #Contrainte : y_i vaut 1 ssi il y a un arc qui sort de la paire i qui est choisi dans le cycle
        @constraint(SP, y[vertice] == sum(x[a] for a in origine[vertice]))
    end
    
    # Contrainte : le cycle doit être de longueur inférieure à L 
    @constraint(SP, sum(x[a] for a in 1:ne(pool)) <= L) 
    
    # On veut un cycle, donc on doit avoir au moins deux sommets
    @constraint(SP, sum(y[i] for i in 1:nv(pool)) >= 2)
    
    # Objectif : maximiser le coût réduit
    @objective(SP,Max,sum( weights[a]*x[a] for a in 1:ne(pool) ))
    

    optimize!(SP)

    # If the subproblem is solved to optimality then return its objective value
    # and the optimal cycle solution
    # Else the pricing problem is infeasible return Inf, []
    if JuMP.termination_status(SP)==MOI.OPTIMAL
        return JuMP.objective_value(SP),JuMP.value.(x)
    else
        return Inf,[]
    end

end

initial_column (generic function with 1 method)

In [1]:
# Génération de colonne bidon 
function init_col(pool)
    added_node = zeros(Int,nv(pool))
    edge_selected = zeros(Int,ne(pool))
    for (ind,arc) in enumerate(edges(pool))
        if added_node[src(arc)] ==0 && added_node[dst(arc)] == 0
            edge_selected[ind]=1
            added_node[src(arc)]=1
            added_node[dst(arc)]=1
        end
    end
    return vcat(-1000,edge_selected)
end

            
        

init_col (generic function with 1 method)